In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,Lambda
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
import re

In [4]:
data = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells."""

In [5]:
sentences = data.split(".")

In [6]:
sentences

['We are about to study the idea of a computational process',
 '\nComputational processes are abstract beings that inhabit computers',
 '\nAs they evolve, processes manipulate other abstract things called data',
 '\nThe evolution of a process is directed by a pattern of rules\ncalled a program',
 ' People create programs to direct processes',
 ' In effect,\nwe conjure the spirits of the computer with our spells',
 '']

In [7]:
clean_sentences = []
for sentence in sentences:
    if sentence == "":
        continue;
    sentence = re.sub('[^A-Za-z0-9]+', ' ', sentence)
    sentence = re.sub(r'(?:^| )\w(?:$| )', ' ',sentence).strip()
    sentence = sentence.lower()
    clean_sentences.append(sentence)

In [8]:
clean_sentences

['we are about to study the idea of computational process',
 'computational processes are abstract beings that inhabit computers',
 'as they evolve processes manipulate other abstract things called data',
 'the evolution of process is directed by pattern of rules called program',
 'people create programs to direct processes',
 'in effect we conjure the spirits of the computer with our spells']

In [9]:
corpus = clean_sentences

In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
print("After converting our words in the corpus into vectors of integers:")
print(sequences)

After converting our words in the corpus into vectors of integers:
[[4, 5, 11, 6, 12, 1, 13, 2, 7, 8], [7, 3, 5, 9, 14, 15, 16, 17], [18, 19, 20, 3, 21, 22, 9, 23, 10, 24], [1, 25, 2, 8, 26, 27, 28, 29, 2, 30, 10, 31], [32, 33, 34, 6, 35, 3], [36, 37, 4, 38, 1, 39, 2, 1, 40, 41, 42, 43]]


In [11]:
word_to_index = {}
index_to_word = {}
for index1,sequence in enumerate(sequences):
    print(sequence)
    words_in_sentence = clean_sentences[index1].split()
    print(words_in_sentence)
    for index2, value in enumerate(sequence):
        index_to_word[value] = words_in_sentence[index2]
        word_to_index[words_in_sentence[index2]] = value

[4, 5, 11, 6, 12, 1, 13, 2, 7, 8]
['we', 'are', 'about', 'to', 'study', 'the', 'idea', 'of', 'computational', 'process']
[7, 3, 5, 9, 14, 15, 16, 17]
['computational', 'processes', 'are', 'abstract', 'beings', 'that', 'inhabit', 'computers']
[18, 19, 20, 3, 21, 22, 9, 23, 10, 24]
['as', 'they', 'evolve', 'processes', 'manipulate', 'other', 'abstract', 'things', 'called', 'data']
[1, 25, 2, 8, 26, 27, 28, 29, 2, 30, 10, 31]
['the', 'evolution', 'of', 'process', 'is', 'directed', 'by', 'pattern', 'of', 'rules', 'called', 'program']
[32, 33, 34, 6, 35, 3]
['people', 'create', 'programs', 'to', 'direct', 'processes']
[36, 37, 4, 38, 1, 39, 2, 1, 40, 41, 42, 43]
['in', 'effect', 'we', 'conjure', 'the', 'spirits', 'of', 'the', 'computer', 'with', 'our', 'spells']


In [12]:
print(index_to_word)
print("------------------------------------------------------------")
print(word_to_index)

{4: 'we', 5: 'are', 11: 'about', 6: 'to', 12: 'study', 1: 'the', 13: 'idea', 2: 'of', 7: 'computational', 8: 'process', 3: 'processes', 9: 'abstract', 14: 'beings', 15: 'that', 16: 'inhabit', 17: 'computers', 18: 'as', 19: 'they', 20: 'evolve', 21: 'manipulate', 22: 'other', 23: 'things', 10: 'called', 24: 'data', 25: 'evolution', 26: 'is', 27: 'directed', 28: 'by', 29: 'pattern', 30: 'rules', 31: 'program', 32: 'people', 33: 'create', 34: 'programs', 35: 'direct', 36: 'in', 37: 'effect', 38: 'conjure', 39: 'spirits', 40: 'computer', 41: 'with', 42: 'our', 43: 'spells'}
------------------------------------------------------------
{'we': 4, 'are': 5, 'about': 11, 'to': 6, 'study': 12, 'the': 1, 'idea': 13, 'of': 2, 'computational': 7, 'process': 8, 'processes': 3, 'abstract': 9, 'beings': 14, 'that': 15, 'inhabit': 16, 'computers': 17, 'as': 18, 'they': 19, 'evolve': 20, 'manipulate': 21, 'other': 22, 'things': 23, 'called': 10, 'data': 24, 'evolution': 25, 'is': 26, 'directed': 27, 'by

In [13]:
vocab_size = len(tokenizer.word_index) + 1
embedding_size = 10
window_size = 2

contexts = []
targets = [] 
for sequence in sequences:
    for i in range(window_size,len(sequence) - window_size):
        context = sequence[i - window_size:i] + sequence[i + 1:i + window_size + 1]
        target = sequence[i]
        contexts.append(context)
        targets.append(target)

In [14]:
for i in range(5):
    words = []
    target = index_to_word.get(targets[i])
    for j in contexts[i]:
        words.append(index_to_word.get(j))
    print(words, "=>", target)

['we', 'are', 'to', 'study'] => about
['are', 'about', 'study', 'the'] => to
['about', 'to', 'the', 'idea'] => study
['to', 'study', 'idea', 'of'] => the
['study', 'the', 'of', 'computational'] => idea


In [15]:
x = np.array(contexts)
y = np.array(targets)

In [16]:
model = Sequential()
model.add(Embedding(input_dim =vocab_size,output_dim=embedding_size,input_length=2*window_size))
model.add(Lambda(lambda x: tf.reduce_mean(x,axis=1)))
model.add(Dense(256,activation='relu'))
model.add(Dense(512,activation='relu'))
model.add(Dense(units=vocab_size,activation='softmax'))

model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

model.fit(x,y,epochs=200,verbose=1)

Epoch 1/200
2/2 [==============================] - 1s 11ms/step - loss: 3.7847 - accuracy: 0.0000e+00
Epoch 2/200
2/2 [==============================] - 0s 2ms/step - loss: 3.7761 - accuracy: 0.2647
Epoch 3/200
2/2 [==============================] - 0s 0s/step - loss: 3.7705 - accuracy: 0.2647
Epoch 4/200
2/2 [==============================] - 0s 16ms/step - loss: 3.7644 - accuracy: 0.2353
Epoch 5/200
2/2 [==============================] - 0s 0s/step - loss: 3.7568 - accuracy: 0.2059
Epoch 6/200
2/2 [==============================] - 0s 17ms/step - loss: 3.7480 - accuracy: 0.2059
Epoch 7/200
2/2 [==============================] - 0s 0s/step - loss: 3.7372 - accuracy: 0.1765
Epoch 8/200
2/2 [==============================] - 0s 0s/step - loss: 3.7249 - accuracy: 0.1765
Epoch 9/200
2/2 [==============================] - 0s 17ms/step - loss: 3.7098 - accuracy: 0.2059
Epoch 10/200
2/2 [==============================] - 0s 0s/step - loss: 3.6923 - accuracy: 0.1765
Epoch 11/200
2/2 [========

2/2 [==============================] - 0s 16ms/step - loss: 1.2840 - accuracy: 0.6765
Epoch 85/200
2/2 [==============================] - 0s 0s/step - loss: 1.2555 - accuracy: 0.6471
Epoch 86/200
2/2 [==============================] - 0s 16ms/step - loss: 1.2374 - accuracy: 0.6471
Epoch 87/200
2/2 [==============================] - 0s 0s/step - loss: 1.2161 - accuracy: 0.6471
Epoch 88/200
2/2 [==============================] - 0s 0s/step - loss: 1.1915 - accuracy: 0.5882
Epoch 89/200
2/2 [==============================] - 0s 0s/step - loss: 1.1624 - accuracy: 0.6176
Epoch 90/200
2/2 [==============================] - 0s 0s/step - loss: 1.1341 - accuracy: 0.6765
Epoch 91/200
2/2 [==============================] - 0s 16ms/step - loss: 1.1131 - accuracy: 0.6176
Epoch 92/200
2/2 [==============================] - 0s 0s/step - loss: 1.0886 - accuracy: 0.6176
Epoch 93/200
2/2 [==============================] - 0s 0s/step - loss: 1.0625 - accuracy: 0.6765
Epoch 94/200
2/2 [===================

2/2 [==============================] - 0s 16ms/step - loss: 0.1579 - accuracy: 1.0000
Epoch 168/200
2/2 [==============================] - 0s 0s/step - loss: 0.1548 - accuracy: 1.0000
Epoch 169/200
2/2 [==============================] - 0s 16ms/step - loss: 0.1525 - accuracy: 1.0000
Epoch 170/200
2/2 [==============================] - 0s 0s/step - loss: 0.1504 - accuracy: 1.0000
Epoch 171/200
2/2 [==============================] - 0s 0s/step - loss: 0.1478 - accuracy: 1.0000
Epoch 172/200
2/2 [==============================] - 0s 16ms/step - loss: 0.1450 - accuracy: 1.0000
Epoch 173/200
2/2 [==============================] - 0s 0s/step - loss: 0.1405 - accuracy: 1.0000
Epoch 174/200
2/2 [==============================] - 0s 16ms/step - loss: 0.1357 - accuracy: 1.0000
Epoch 175/200
2/2 [==============================] - 0s 0s/step - loss: 0.1310 - accuracy: 1.0000
Epoch 176/200
2/2 [==============================] - 0s 0s/step - loss: 0.1267 - accuracy: 1.0000
Epoch 177/200
2/2 [=======

In [17]:
embeddings = model.get_weights()[0]

pca = PCA(n_components=2)
reduced_embeddings = pca.fit_transform(embeddings)

In [18]:
test_sentences = [
    "we are to study",
    "create programs direct processes",
    "spirits process study program",
    "idea study people create"
]

In [19]:
for test_sentence in test_sentences:
    test_words = test_sentence.split()
    print("words:",test_words)
    xtest = []
    for i in test_words:
        xtest.append(word_to_index.get(i))
    xtest = np.array([xtest])
    print("Indexes:", xtest)
    test_predictions = model.predict(xtest)
    ypred = np.argmax(test_predictions[0])
    print("Predictions: ",test_words, "=>",index_to_word.get(ypred))
    print("\n")

words: ['we', 'are', 'to', 'study']
Indexes: [[ 4  5  6 12]]
1/1 [==============================] - 0s 63ms/step
Predictions:  ['we', 'are', 'to', 'study'] => about


words: ['create', 'programs', 'direct', 'processes']
Indexes: [[33 34 35  3]]
1/1 [==============================] - 0s 27ms/step
Predictions:  ['create', 'programs', 'direct', 'processes'] => to


words: ['spirits', 'process', 'study', 'program']
Indexes: [[39  8 12 31]]
1/1 [==============================] - 0s 25ms/step
Predictions:  ['spirits', 'process', 'study', 'program'] => spirits


words: ['idea', 'study', 'people', 'create']
Indexes: [[13 12 32 33]]
1/1 [==============================] - 0s 21ms/step
Predictions:  ['idea', 'study', 'people', 'create'] => programs




In [20]:
import random

In [23]:
text_sentences = []
for i in range(5):
    n = random.randint(0,9)
    text_sentences.append(index_to_word.get(contexts[n]))

TypeError: unhashable type: 'list'

In [22]:
text_sentences

[[3, 5, 14, 15], [1, 13, 7, 8], [12, 1, 2, 7], [6, 12, 13, 2], [7, 3, 9, 14]]